In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

class PdScorer:
    def __init__(self, fitted_model):
        self.model = fitted_model
        self.grid_scores = self.create_grid_scores()

    def create_grid_scores(self):
        # Extraction des coefficients estimés du modèle
        coefficients = self.model.params

        # Normalisation des coefficients pour les rendre comparables
        normalized_coefficients = (coefficients - coefficients.min()) / (coefficients.max() - coefficients.min())

        # Création d'une grille de score sur mille basée sur les coefficients normalisés
        grid_scores = np.round(normalized_coefficients * 1000).astype(int)

        return grid_scores

    def get_grid_scores(self):
        return self.grid_scores

    def transform(self, data, target_col):
        # Ajoute une colonne constante pour intercept dans data

        # Extraction de la variable cible (target)
        y = data[target_col]

        # Suppression de la variable cible du DataFrame
        data = data.drop(columns=[target_col])

        # Calcul des scores normalisés sur mille en utilisant les coefficients normalisés
        normalized_scores = np.dot(data, self.grid_scores[1:]) + self.grid_scores[0]

        # Transformation des scores normalisés en scores sur 1000
        scores_on_thousand = np.round((normalized_scores - normalized_scores.min()) / (normalized_scores.max() - normalized_scores.min()) * 1000).astype(int)

        return scores_on_thousand



In [6]:
# Exemple d'utilisation
# Génération de données d'exemple (remplacez X et y par vos propres données)
np.random.seed(42)
data1 = pd.DataFrame({
    'Category1': np.random.choice(['A', 'B', 'C'], size=100),
    'Category2': np.random.choice(['X', 'Y', 'Z'], size=100),
    'Variable1': np.random.rand(100),
    'Variable2': np.random.rand(100),
    'Target': np.random.choice([0, 1], size=100),
})

data2 = pd.DataFrame({
    'Category1': np.random.choice(['A', 'B', 'C'], size=50),
    'Category2': np.random.choice(['X', 'Y', 'Z'], size=50),
    'Variable1': np.random.rand(50),
    'Variable2': np.random.rand(50),
    'Target': np.random.choice([0, 1], size=50),
})

# Spécification de la formule pour le modèle logistique
formula = 'Target ~ Category1 + Category2 + Variable1 + Variable2'

# Estimation du modèle de régression logistique avec statsmodels
logit_model = sm.Logit.from_formula(formula, data1)
fitted_model = logit_model.fit(disp=False)

In [7]:
print(fitted_model.summary())

                           Logit Regression Results                           
Dep. Variable:                 Target   No. Observations:                  100
Model:                          Logit   Df Residuals:                       93
Method:                           MLE   Df Model:                            6
Date:                Sun, 18 Feb 2024   Pseudo R-squ.:                 0.04872
Time:                        23:56:33   Log-Likelihood:                -65.633
converged:                       True   LL-Null:                       -68.994
Covariance Type:            nonrobust   LLR p-value:                    0.3472
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.9446      0.670      1.410      0.159      -0.369       2.258
Category1[T.B]    -0.2421      0.505     -0.479      0.632      -1.232       0.748
Category1[T.C]     0.4621      0.528

In [8]:
# Instanciation de la classe PdScorer avec le modèle ajusté
pd_scorer = PdScorer(fitted_model)

# Utilisation de la méthode get_grid_scores pour obtenir la grille de score
grid_scores = pd_scorer.get_grid_scores()
print("Grille de score sur mille:")
print(grid_scores)

# Utilisation de la méthode transform pour obtenir les scores sur 1000 pour data1
# Assurez-vous que les noms de colonnes dans 'data1' correspondent à ceux utilisés dans le modèle
scores_data1 = pd_scorer.transform(data1, 'Target')
# Affichage des 10 premiers scores pour data1
print("\nScores sur 1000 pour les 10 premières lignes de data1:")
print(scores_data1[:10])

# Utilisation de la méthode transform pour obtenir les scores sur 1000 pour data2
# Assurez-vous que les noms de colonnes dans 'data2' correspondent à ceux utilisés dans le modèle
scores_data2 = pd_scorer.transform(data2, 'Target')
# Affichage des 10 premiers scores pour data2
print("\nScores sur 1000 pour les 10 premières lignes de data2:")
print(scores_data2[:10])

Grille de score sur mille:
Intercept         1000
Category1[T.B]     388
Category1[T.C]     751
Category2[T.Y]     433
Category2[T.Z]     154
Variable1            0
Variable2          568
dtype: int32


ValueError: shapes (100,5) and (6,) not aligned: 5 (dim 1) != 6 (dim 0)